# LSTM Train - Kaza - No PCA

In [1]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader, random_split

In [ ]:
DATASET_PATH = config.DATA_INTERIM
jsonl_file = DATASET_PATH / "data.jsonl"
if jsonl_file.exists():
    df = pd.read_json(jsonl_file, lines=True)
    if "traffic_sequence" in df.columns:
        df = pd.DataFrame(df["traffic_sequence"].tolist())
else:
    df = pd.read_parquet(DATASET_PATH)
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [4]:
# PCA kullanılmıyor
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
train_len = int(len(dataset) * 0.8)
val_len = len(dataset) - train_len
train_ds, val_ds = random_split(dataset, [train_len, val_len])
train_loader = DataLoader(train_ds, batch_size=1000000, shuffle=True, num_workers=8)
val_loader = DataLoader(val_ds, batch_size=1000000, num_workers=8)
predictor = TrafficPredictor('lstm', dataset.sensors, X_STEP, Y_STEP, pca_model=dataset.pca if False else None)
predictor.train(train_loader, val_loader, epochs=1000, save_path=config.PROJECT_ROOT.joinpath('models/lstm_kaza_nopca.pt'), save_optimizer=True)
predictor.show()

Epoch 1/1000  Train Loss: 27902.7812  Val Loss: 27962.9004  Val MSE: 27962.9023  Val R2: -0.1774
Epoch 2/1000  Train Loss: 27895.5000  Val Loss: 27955.0098  Val MSE: 27955.0098  Val R2: -0.1771
Epoch 3/1000  Train Loss: 27887.6094  Val Loss: 27946.5957  Val MSE: 27946.5977  Val R2: -0.1767
Epoch 4/1000  Train Loss: 27879.2012  Val Loss: 27937.7363  Val MSE: 27937.7402  Val R2: -0.1764
Epoch 5/1000  Train Loss: 27870.3438  Val Loss: 27928.5117  Val MSE: 27928.5137  Val R2: -0.1760
Epoch 6/1000  Train Loss: 27861.1211  Val Loss: 27918.9824  Val MSE: 27918.9824  Val R2: -0.1756
Epoch 7/1000  Train Loss: 27851.5898  Val Loss: 27909.2090  Val MSE: 27909.2051  Val R2: -0.1752
Epoch 8/1000  Train Loss: 27841.8184  Val Loss: 27899.2441  Val MSE: 27899.2441  Val R2: -0.1748
Epoch 9/1000  Train Loss: 27831.8555  Val Loss: 27889.1270  Val MSE: 27889.1250  Val R2: -0.1743
Epoch 10/1000  Train Loss: 27821.7402  Val Loss: 27878.9082  Val MSE: 27878.9082  Val R2: -0.1739
Epoch 11/1000  Train Loss: 27

KeyboardInterrupt: 